In [1]:
import feedparser
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
import folium
import os
%matplotlib notebook
from sqlalchemy import create_engine
import psycopg2 
from io import BytesIO as StringIO
from datetime import datetime, timedelta


/Users/michael/GEO/adampy2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


In [22]:
#Get RSS
feed = 'https://emergency.copernicus.eu/mapping/list-of-components/EMSR348/feed'
gfds = 'http://www.gdacs.org/flooddetection/data.aspx?from=20190323&to=20190323&type=rss&alertlevel=red&datatype=4DAYS'
EMS = 'http://emergency.copernicus.eu/mapping/activations-rapid/feed' #Filter flood #filter disastertype (flood)
gdacs = 'http://www.gdacs.org/xml/rss.xml'
eo = 'https://earthobservatory.nasa.gov/feeds/earth-observatory.rss'
ptc = 'https://ptwc.weather.gov/feeds/ptwc_rss_pacific.xml'


## GDACS

In [3]:
GDACS_RSS = 'http://www.gdacs.org/xml/rss.xml'

GDACS_URL = [GDACS_RSS]

gdacs_feeds = []
gdacs_iso3 = []
gdacs_country = []
gdacs_etype = []
gdacs_epid = []
gdacs_evid = []
gdacs_guid = []
gdacs_pubdate = []
gdacs_bbox = []
gdacs_coords = []
gdacs_alevel = []
gdacs_ascore = []

wfpiso3 = ['TUR','CHL', 'IDN', 'PHL', 'SDN', 'SSD', 'DRC', 'BGD', 'CAF', 
           'BWA', 'TCD', 'KHM', 'IND', 'MYS','ZWE','MDG','MOZ']

def get_gdacs():
    for url in GDACS_URL:
        gdacs_feeds.append(feedparser.parse(url))

    for feed in gdacs_feeds:
        for post in feed.entries:
            gdacs_iso3.append(post.gdacs_iso3)
            gdacs_country.append(post.gdacs_country)
            gdacs_etype.append(post.gdacs_eventtype)
            gdacs_epid.append(post.gdacs_episodeid)
            gdacs_evid.append(post.gdacs_eventid)
            gdacs_guid.append(post.guid)
            gdacs_pubdate.append(post.published)
            gdacs_bbox.append(post.gdacs_bbox)
            gdacs_coords.append(post.where)
            gdacs_alevel.append(post.gdacs_episodealertlevel)
            gdacs_ascore.append(post.gdacs_alertscore)
            
        guiddf = pd.DataFrame(gdacs_guid)       #0
        pubdatedf = pd.DataFrame(gdacs_pubdate) #1
        isodf = pd.DataFrame(gdacs_iso3)        #2   
        countrydf = pd.DataFrame(gdacs_country) #3
        etypedf = pd.DataFrame(gdacs_etype)     #4
        epidf = pd.DataFrame(gdacs_epid)        #5
        evidf = pd.DataFrame(gdacs_evid)        #6
        bboxdf = pd.DataFrame(gdacs_bbox)       #7
        coordf = pd.DataFrame(gdacs_coords)     #8 #9
        aleveldf = pd.DataFrame(gdacs_alevel)   #10
        ascoredf = pd.DataFrame(gdacs_ascore)   #11
        
        frames = [guiddf, pubdatedf, isodf, countrydf, etypedf, epidf, evidf, bboxdf,coordf, aleveldf, ascoredf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        #return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        return df



## RELIEF WEB Int

In [4]:
RF_RSS ='https://reliefweb.int/disasters/rss.xml'

RF_URL = [RF_RSS]

rw_feeds = []
rw_pub = []
rw_title = []
rw_category = []
rw_desc = []


def get_rw():
    for url in RF_URL:
        rw_feeds.append(feedparser.parse(url))

    for feed in rw_feeds:
        for post in feed.entries:
            rw_pub.append(post.published)
            rw_title.append(post.title)
            rw_category.append(post.category)
            rw_desc.append(post.description)
            
            
        titledf = pd.DataFrame(rw_title)       #0
        pubdatedf = pd.DataFrame(rw_pub) #1
        catdf = pd.DataFrame(rw_category)        #2   
        descdf = pd.DataFrame(rw_desc)
        
        frames = [titledf,pubdatedf, catdf, descdf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        #return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        return df

## Copernicus

In [5]:
EMS_RSS = 'http://emergency.copernicus.eu/mapping/activations-rapid/feed'

EMS_URL = [EMS_RSS]

ems_guid = []
ems_title = []
ems_cat = []
ems_geo = []
ems_pub = []
ems_efeeds = []

wfpiso3 = ['TUR','CHL', 'IDN', 'PHL', 'SDN', 'SSD', 'DRC', 'BGD', 'CAF', 
           'BWA', 'TCD', 'KHM', 'IND', 'MYS','ZWE','MDG','MOZ']

def get_ems():
    for url in EMS_URL:
        ems_efeeds.append(feedparser.parse(url))

    for feed in ems_efeeds:
        for post in feed.entries:
            ems_guid.append(post.guid)
            ems_title.append(post.title)
            ems_cat.append(post.category)
            ems_geo.append(post.where)
            ems_pub.append(post.published)
            
        guidf = pd.DataFrame(ems_guid)   #0
        titledf = pd.DataFrame(ems_title)  #1
        catdf= pd.DataFrame(ems_cat)     #2
        geodf= pd.DataFrame(ems_geo)     #3
        pubdf= pd.DataFrame(ems_pub)     #4
        
        frames = [guidf, titledf, catdf, geodf, pubdf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        #return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        #return df.where[df['category'] == 'FL']
        return df
    

## Global Flood Detection System

In [6]:
GFDS_RSS = 'http://www.gdacs.org/flooddetection/data.aspx?from=20190323&to=20190323&type=rss&alertlevel=red&datatype=7DAYS'

GFDS_URL = [GFDS_RSS]

gfds_desc = []
gfds_title = []
gfds_country = []
gfds_geo = []
gfds_magnitude = []
gfds_pub = []
gfds_feeds = []

wfpiso3 = ['TUR','CHL', 'IDN', 'PHL', 'SDN', 'SSD', 'DRC', 'BGD', 'CAF', 
           'BWA', 'TCD', 'KHM', 'IND', 'MYS','ZWE','MDG','MOZ']

def get_gfds():
    for url in GFDS_URL:
        gfds_feeds.append(feedparser.parse(url))

    for feed in gfds_feeds:
        for post in feed.entries:
            gfds_desc.append(post.description)
            gfds_title.append(post.title)
            gfds_country.append(post.country)
            gfds_geo.append(post.where)
            gfds_magnitude.append(post.magnitude)
            #pub.append(post.record_date)
            
        descdf = pd.DataFrame(gfds_desc)   #0
        titledf = pd.DataFrame(gfds_title)  #1
        countrydf= pd.DataFrame(gfds_country)     #2
        geodf= pd.DataFrame(gfds_geo)     #3
        magdf = pd.DataFrame(gfds_magnitude)
        #pubdf= pd.DataFrame(pub)     #4
        
        frames = [descdf,titledf,countrydf,geodf, magdf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        #return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        #return df.where[df['category'] == 'FL']
        return df
    

## GET DATA

In [19]:
gdacs_data = get_gdacs()
ems_data = get_ems()
gfds_data = get_gfds()
rw_data = get_rw()

reliefweb_f = rw_data.loc[rw_data[2] == 'Flash Flood']
gdacs_fl = gdacs_data.loc[gdacs_data[4] == 'FL']
gdacs_tc = gdacs_data.loc[gdacs_data[4] == 'TC']
ems_f = ems_data.loc[ems_data[2] == 'Flood']
ems_s = ems_data.loc[ems_data[2] == 'Storm']

ems_f

,0,1,2,3,4,5
0,EMSR352,[EMSR352] Flood in Iran,Flood,"(48.72481, 31.38581)",Point,"Tue, 09 Apr 2019 17:21:26 +0200"
2,EMSR349,"[EMSR349] Flood and Wind Storm in Manicaland, ...",Flood,"(32.03, -19.87)",Point,"Mon, 18 Mar 2019 01:51:55 +0100"
9,EMSR342,"[EMSR342] Flood in Townsville, Australia",Flood,"(146.73, -19.32)",Point,"Fri, 01 Feb 2019 06:09:34 +0100"
13,EMSR337,[EMSR337] Flood in Western Greece,Flood,"(21.67, 37.58)",Point,"Mon, 14 Jan 2019 14:22:16 +0100"
17,EMSR333,"[EMSR333] Flood in Sicily, Italy",Flood,"(13.176, 37.841)",Point,"Sun, 04 Nov 2018 23:16:24 +0100"
18,EMSR332,"[EMSR332] Flood in Veneto, Italy",Flood,"(11.84376, 45.71602)",Point,"Sat, 03 Nov 2018 00:48:54 +0100"
20,EMSR330,"[EMSR330] Flood in Sicily, Italy",Flood,"(14.70709, 37.28147)",Point,"Thu, 25 Oct 2018 19:12:24 +0200"
21,EMSR329,"[EMSR329] Flood in southern Sardinia, Italy",Flood,"(9.01, 39.25)",Point,"Wed, 24 Oct 2018 17:53:44 +0200"
23,EMSR327,"[EMSR327] Flash Flood in Andalusia, Spain",Flood,"(-5.08, 37.54)",Point,"Tue, 23 Oct 2018 21:06:06 +0200"
24,EMSR326,"[EMSR326] Flash Flood in Malaga, Spain",Flood,"(-4.97, 36.88)",Point,"Sun, 21 Oct 2018 14:34:06 +0200"


In [9]:
#EMS

geometry = [Point(xy) for xy in zip(ems_f[3])]
emsdf = ems_f.drop([3], axis=1)
crs = {'init': 'epsg:4326'}
ems_gdf = GeoDataFrame(emsdf, crs=crs, geometry=geometry)
ems_gdf.head()

,0,1,2,4,5,geometry
0,EMSR352,[EMSR352] Flood in Iran,Flood,Point,"Tue, 09 Apr 2019 17:21:26 +0200",POINT (48.72481 31.38581)
2,EMSR349,"[EMSR349] Flood and Wind Storm in Manicaland, ...",Flood,Point,"Mon, 18 Mar 2019 01:51:55 +0100",POINT (32.03 -19.87)
9,EMSR342,"[EMSR342] Flood in Townsville, Australia",Flood,Point,"Fri, 01 Feb 2019 06:09:34 +0100",POINT (146.73 -19.32)
13,EMSR337,[EMSR337] Flood in Western Greece,Flood,Point,"Mon, 14 Jan 2019 14:22:16 +0100",POINT (21.67 37.58)
17,EMSR333,"[EMSR333] Flood in Sicily, Italy",Flood,Point,"Sun, 04 Nov 2018 23:16:24 +0100",POINT (13.176 37.841)


In [10]:
#GDACS

geometry = [Point(xy) for xy in zip(gdacs_data[8])]
gdacsdf = gdacs_data.drop([8], axis=1)
crs = {'init': 'epsg:4326'}
gdacs_gdf = GeoDataFrame(gdacsdf, crs=crs, geometry=geometry)
gdacs_gdf.head()

,0,1,2,3,4,5,6,7,9,10,11,geometry
0,EQ1176991,"Thu, 18 Apr 2019 14:46:01 GMT",,Western Indian-Antarctic Ridge,EQ,1257029,1176991,135.4904 143.4904 -55.0555 -47.0555,Point,Green,0.01,POINT (139.4904 -51.0555)
1,EQ1176951,"Thu, 18 Apr 2019 05:01:06 GMT",CHN,China,EQ,1256985,1176951,117.6934 125.6934 19.9888 27.9888,Point,Green,0.0318797790530688,POINT (121.6934 23.9888)
2,EQ1176950,"Thu, 18 Apr 2019 04:06:34 GMT",CHL,Chile,EQ,1256973,1176950,-75.6087 -67.6087 -33.3707 -25.3707,Point,Green,0.01,POINT (-71.6087 -29.3707)
3,EQ1176800,"Tue, 16 Apr 2019 16:59:16 GMT",SGS,South Georgia and the South Sandwich Islands,EQ,1256784,1176800,-27.5963 -19.5963 -62.7417 -54.7417,Point,Green,0.01,POINT (-23.5963 -58.7417)
4,EQ1176793,"Tue, 16 Apr 2019 16:01:16 GMT",,South Of Kermadec Islands,EQ,1256769,1176793,-183.5758 -175.5758 -36.3695 -28.3695,Point,Green,0.01,POINT (-179.5758 -32.3695)


In [20]:
geometry = [Point(xy) for xy in zip(ems_data[3])]
emsdf = ems_data.drop([3], axis=1)
crs = {'init': 'epsg:4326'}
ems_gdf = GeoDataFrame(emsdf, crs=crs, geometry=geometry)
ems_gdf.head()

,0,1,2,4,5,geometry
0,EMSR352,[EMSR352] Flood in Iran,Flood,Point,"Tue, 09 Apr 2019 17:21:26 +0200",POINT (48.72481 31.38581)
1,EMSR350,[EMSR350] Trevor and Veronica tropical cyclone...,Storm,Point,"Sat, 23 Mar 2019 15:29:05 +0100",POINT (136.4 -16)
2,EMSR349,"[EMSR349] Flood and Wind Storm in Manicaland, ...",Flood,Point,"Mon, 18 Mar 2019 01:51:55 +0100",POINT (32.03 -19.87)
3,EMSR348,[EMSR348] Tropical Cyclone IDAI in Mozambique,Storm,Point,"Fri, 15 Mar 2019 10:56:55 +0100",POINT (35.5 -18.9)
4,EMSR347,[EMSR347] Floods in Malawi,Storm,Point,"Sun, 10 Mar 2019 16:17:23 +0100",POINT (35.25515 -14.55252)


In [17]:
#aoi = gdacsjson.loc[gdacsjson['alertlevelepisode'] == 'Green']

m = folium.Map(tiles='stamentoner') #cartodbpositron #stamentoner #cartodbdark_matter
#folium.GeoJson(ems_gdf).add_to(m)
#folium.GeoJson(gdacs_gdf).add_to(m)
folium.GeoJson(gdacs_gdf).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [21]:
root = "http://www.gdacs.org/datareport/resources/"
eventTC = "TC/"
eventDR = "DR/"
eventEQ = "EQ/"
eventID = "1000533"
objID = "/geojson_"
objNum = "_1"
objForm = ".geojson"

tceventtest = root + eventTC + eventID + objID + eventID + objNum + objForm

print(tceventtest)

http://www.gdacs.org/datareport/resources/TC/1000533/geojson_1000533_1.geojson


In [10]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
api = SentinelAPI('mgmanalili.updge', 'Suite#41', 'https://scihub.copernicus.eu/dhus')


In [12]:
#officers = '/home/michael/Desktop/gis_pips.csv'
#gis = pd.read_csv(officers)
#rec = gis[(gis['Regional Bureau'] == 'RBN') | (gis['Regional Bureau'] == 'RBJ') | (gis['Regional Bureau'] == 'RBB')]